### Xóa DB trên Neo4j Browser trước nếu có

In [1]:
from neo4j import GraphDatabase

# Cấu hình kết nối
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

def wipe_everything():
    print("🚀 Bắt đầu quy trình xóa sổ toàn bộ dữ liệu...")
    driver = GraphDatabase.driver(URI, auth=AUTH)
    
    with driver.session() as session:
        # 1. Xóa toàn bộ Constraints (Ràng buộc)
        print("   - Đang tìm và xóa Constraints...")
        constraints = session.run("SHOW CONSTRAINTS YIELD name")
        for record in constraints:
            name = record["name"]
            try:
                session.run(f"DROP CONSTRAINT {name}")
                print(f"     Deleted constraint: {name}")
            except Exception as e:
                print(f"     Skipped/Error {name}: {e}")

        # 2. Xóa toàn bộ Indexes (Chỉ mục)
        print("   - Đang tìm và xóa Indexes...")
        indexes = session.run("SHOW INDEXES YIELD name, type WHERE type <> 'LOOKUP'") 
        # LOOKUP index là mặc định của hệ thống, không xóa được và không nên xóa
        for record in indexes:
            name = record["name"]
            try:
                session.run(f"DROP INDEX {name}")
                print(f"     Deleted index: {name}")
            except Exception as e:
                print(f"     Skipped/Error {name}: {e}")

        # 3. Xóa sạch Node và Relationship (Dữ liệu)
        print("   - Đang xóa sạch Node và Relationship...")
        # Với 5000 node, lệnh này chạy vèo cái là xong, không cần chia batch
        session.run("MATCH (n) DETACH DELETE n")
        
    driver.close()
    print("✨ XONG! Database giờ trắng trơn như tờ giấy.")

def verify_empty():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) AS count")
        count = result.single()["count"]
        print(f"🔍 Kiểm tra lại: Số lượng node hiện tại là: {count}")
    driver.close()

if __name__ == "__main__":
    wipe_everything()
    verify_empty()

🚀 Bắt đầu quy trình xóa sổ toàn bộ dữ liệu...
   - Đang tìm và xóa Constraints...
     Deleted constraint: constraint_1e6bab2b
     Deleted constraint: constraint_43f9c6cd
     Deleted constraint: constraint_4b2945cd
     Deleted constraint: constraint_82b3e63a
     Deleted constraint: constraint_e578272a
   - Đang tìm và xóa Indexes...
     Deleted index: general_info_index
     Deleted index: searchable_index
     Deleted index: stopNameIndex
     Deleted index: stop_id_idx
     Deleted index: stop_name_idx
   - Đang xóa sạch Node và Relationship...
✨ XONG! Database giờ trắng trơn như tờ giấy.
🔍 Kiểm tra lại: Số lượng node hiện tại là: 0


## Push data bus lên neo4j

In [2]:
import json
import time
from neo4j import GraphDatabase
from math import radians, cos, sin, asin, sqrt

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw") 
INPUT_FILE = "../data/hcm_bus_final.json"

def haversine(lon1, lat1, lon2, lat2):
    """Tính khoảng cách giữa 2 tọa độ (km)"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r

def import_data():
    print(f"🚀 BẮT ĐẦU NẠP DỮ LIỆU TỪ '{INPUT_FILE}'...")
    
    # 1. Đọc file JSON
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy file '{INPUT_FILE}'. Hãy kiểm tra lại tên file.")
        return

    routes = data['Routes']
    stops = data['UniqueStops']
    
    # Cache tọa độ để tính toán nhanh
    stop_coords_map = {s['StopId']: {'lat': s['Lat'], 'lng': s['Lng']} for s in stops}

    driver = GraphDatabase.driver(URI, auth=AUTH)
    
    with driver.session() as session:
        # --- BƯỚC 1: XÓA SẠCH SẼ DB CŨ ---
        print("🧹 Đang xóa sạch dữ liệu cũ (Reset)...")
        session.run("MATCH (n) DETACH DELETE n")
        
        # --- BƯỚC 2: TẠO INDEX (ĐỂ TÌM KIẾM NHANH) ---
        print("⚙️ Đang tạo Index tối ưu...")
        session.run("CREATE INDEX stop_id_idx IF NOT EXISTS FOR (s:BusStop) ON (s.id)")
        session.run("CREATE INDEX stop_name_idx IF NOT EXISTS FOR (s:BusStop) ON (s.name)")
        # Tạo Fulltext Index cho tính năng tìm kiếm thông minh
        session.run("CREATE FULLTEXT INDEX stopNameIndex IF NOT EXISTS FOR (n:BusStop) ON EACH [n.name, n.code, n.search]")

        # --- BƯỚC 3: NẠP TRẠM (NODES) ---
        print(f"📍 Đang tạo {len(stops)} Trạm dừng...")
        batch_size = 2000
        for i in range(0, len(stops), batch_size):
            batch = stops[i:i+batch_size]
            session.run("""
            UNWIND $batch AS s
            CREATE (n:BusStop {
                id: s.StopId,
                name: s.Name,
                code: s.Code,
                lat: s.Lat, 
                lng: s.Lng,
                search: s.Search,
                street: s.Street
            })
            """, batch=batch)

        # --- BƯỚC 4: NẠP TUYẾN & QUAN HỆ (PHẦN QUAN TRỌNG NHẤT) ---
        print(f"🚌 Đang xử lý {len(routes)} Tuyến xe...")
        
        edges_batch = [] 
        
        for r in routes:
            # Tạo Node Tuyến Xe
            display_name = f"Bus {r['RouteNo']}: {r['RouteName']}"
            price_clean = r['Tickets'].replace("<br/>", "\n").replace("&nbsp", " ") if r['Tickets'] else ""
            
            session.run("""
            MERGE (r:BusRoute {id: $rid})
            SET r.route_no = $rno, r.name = $display, r.price = $price
            """, rid=r['RouteId'], rno=r['RouteNo'], display=display_name, price=price_clean)

            route_stops = r['Stops']
            if not route_stops: continue

            # A. Nối Trạm vào Tuyến (ON_ROUTE) - FIX LỖI GHI ĐÈ
            # Quan trọng: Thêm {direction: ...} vào trong MERGE để tách biệt chiều đi/về
            session.run("""
            UNWIND $stops AS s
            MATCH (r:BusRoute {id: $rid})
            MATCH (st:BusStop {id: s.StopId})
            MERGE (st)-[rel:ON_ROUTE {direction: s.Direction}]->(r)
            SET rel.order = s.Order
            """, stops=route_stops, rid=r['RouteId'])

            # B. Nối Trạm với Trạm (NEXT_HOP) - Để vẽ đường đi
            for i in range(len(route_stops) - 1):
                curr = route_stops[i]
                next_s = route_stops[i+1]
                
                # Chỉ nối nếu cùng một chiều (Direction giống nhau)
                if curr['Direction'] == next_s['Direction']:
                    c_id = curr['StopId']
                    n_id = next_s['StopId']
                    
                    if c_id in stop_coords_map and n_id in stop_coords_map:
                        c_coords = stop_coords_map[c_id]
                        n_coords = stop_coords_map[n_id]
                        
                        dist_km = haversine(c_coords['lng'], c_coords['lat'], n_coords['lng'], n_coords['lat'])
                        time_min = (dist_km / 30) * 60 # Giả sử tốc độ 30km/h
                        if time_min < 0.5: time_min = 0.5
                        
                        edges_batch.append({
                            "from_id": c_id, "to_id": n_id, "route_id": r['RouteId'],
                            "dist": dist_km, "time": time_min,
                            "direction": curr['Direction'] # Lưu luôn chiều vào cạnh nối
                        })

            # Nạp Batch NEXT_HOP
            if len(edges_batch) >= 2000:
                session.run("""
                UNWIND $edges AS e
                MATCH (a:BusStop {id: e.from_id})
                MATCH (b:BusStop {id: e.to_id})
                MERGE (a)-[r:NEXT_HOP {route_id: e.route_id, direction: e.direction}]->(b)
                SET r.dist = e.dist, r.time = e.time
                """, edges=edges_batch)
                edges_batch = []
        
        # Nạp nốt số dư
        if edges_batch:
            session.run("""
            UNWIND $edges AS e
            MATCH (a:BusStop {id: e.from_id})
            MATCH (b:BusStop {id: e.to_id})
            MERGE (a)-[r:NEXT_HOP {route_id: e.route_id, direction: e.direction}]->(b)
            SET r.dist = e.dist, r.time = e.time
            """, edges=edges_batch)

    driver.close()
    print("\n✅ THÀNH CÔNG! Dữ liệu đã được nạp chuẩn chỉnh.")

if __name__ == "__main__":
    import_data()

🚀 BẮT ĐẦU NẠP DỮ LIỆU TỪ '../data/hcm_bus_final.json'...
🧹 Đang xóa sạch dữ liệu cũ (Reset)...
⚙️ Đang tạo Index tối ưu...
📍 Đang tạo 5062 Trạm dừng...
🚌 Đang xử lý 162 Tuyến xe...

✅ THÀNH CÔNG! Dữ liệu đã được nạp chuẩn chỉnh.


## Push data văn hóa du lịch

In [3]:
import json
import re
import time
from neo4j import GraphDatabase
from datetime import datetime

# ================= CẤU HÌNH =================
# Lưu ý: Đảm bảo đường dẫn file chính xác so với vị trí file code của bạn
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")
JSON_PATH = "../data/VanHoaVaDuLichVN_Full_Img.json" 

# Mapping Label (Chuẩn hóa tên Label trong Neo4j)
LABEL_MAP = {
    "Place": "TouristDestination",
    "Festival": "CulturalEvent",
    "Religious": "SpiritualSite",
    "Nature": "NaturalLandscape",
    "GeneralTopic": "GeneralInfo",
    "Cuisine": "Culinary"
}

def clean_text(text):
    """Làm sạch dữ liệu text, xử lý null"""
    return str(text).strip() if text else ""

def generate_spot_id(parent_code, item_name):
    """Tạo ID duy nhất: PARENT_CODE + ITEM_NAME"""
    p_code = clean_text(parent_code).replace("_", "").upper()
    n_clean = re.sub(r'[^\w]', '', clean_text(item_name).upper())
    return f"SPOT_{p_code}_{n_clean}"

def setup_constraints(session):
    """
    Tạo Constraint Unique. 
    Sử dụng try-except để bỏ qua lỗi nếu Constraint đã tồn tại (Fix lỗi IF NOT EXISTS)
    """
    print("⚙️  Đang cấu hình Constraints (Ràng buộc dữ liệu)...")
    
    # Danh sách các lệnh tạo constraint chuẩn cho Neo4j 5.x/Aura
    constraints = [
        "CREATE CONSTRAINT FOR (c:Country) REQUIRE c.id IS UNIQUE",
        "CREATE CONSTRAINT FOR (p:Province) REQUIRE p.id IS UNIQUE",
        "CREATE CONSTRAINT FOR (n:Searchable) REQUIRE n.id IS UNIQUE",
        "CREATE CONSTRAINT FOR (g:GeneralInfo) REQUIRE g.id IS UNIQUE"
    ]

    for query in constraints:
        try:
            session.run(query)
            print(f"   ✅ Đã thực thi: {query}")
        except Exception as e:
            # Nếu lỗi chứa "already exists" thì bỏ qua, còn lỗi khác thì in ra cảnh báo
            error_msg = str(e).lower()
            if "already exists" in error_msg or "equivalent constraint" in error_msg:
                pass 
            else:
                print(f"   ⚠️ Cảnh báo (có thể bỏ qua nếu đã có): {e}")

# --- HÀM 1: IMPORT GENERAL INFO (CẤP QUỐC GIA) ---
def import_general_info(tx, batch_data):
    query = """
    UNWIND $batch AS row
    
    // 1. Tạo Node QUỐC GIA (Country) - Chỉ có 1 node duy nhất
    MERGE (c:Country {id: "VN"})
    SET c.name = "VIETNAM",
        c.type = "National",
        c.last_updated = datetime()
    
    // 2. Tạo Node thông tin chung
    MERGE (n:GeneralInfo {id: row.id})
    SET n.name = row.name,
        n.original_type = row.original_type,
        n.content = row.content,
        n.embedding_text = row.embedding_text, // Giữ nguyên gốc
        n.image_path = row.image_path,
        n.last_updated = datetime()
    
    // 3. Nối quan hệ: GeneralInfo -> THUỘC VỀ -> Country
    MERGE (n)-[:BELONGS_TO]->(c)
    """
    tx.run(query, batch=batch_data)

# --- HÀM 2: IMPORT PROVINCES (CẤP TỈNH) ---
def import_provinces(tx, batch_data):
    query = """
    UNWIND $batch AS row
    
    MERGE (p:Province {id: row.province_id})
    SET p.name = row.province_name,
        p.last_updated = datetime()
    
    MERGE (n:Searchable {id: row.id})
    SET n.name = row.name,
        n.province_name = row.province_name,
        n.original_type = row.original_type,
        n.content = row.content,
        n.embedding_text = row.embedding_text,
        n.image_path = row.image_path,
        n.last_updated = datetime()

    WITH n, p, row
    CALL apoc.create.addLabels(n, [row.neo4j_label]) YIELD node
    
    MERGE (n)-[:LOCATED_IN]->(p)
    """
    tx.run(query, batch=batch_data)

def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)
    print(f"🚀 Bắt đầu xử lý file: {JSON_PATH}")

    # --- 1. ĐỌC FILE JSON ---
    try:
        with open(JSON_PATH, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy file tại '{JSON_PATH}'. Hãy kiểm tra lại đường dẫn.")
        return
    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")
        return

    # --- 2. CHUẨN BỊ DỮ LIỆU ---
    
    # A. Dữ liệu General Information
    general_infos = data.get("General_Information", [])
    print(f"🔹 Tìm thấy {len(general_infos)} mục General Information.")
    
    gen_nodes = []
    for item in general_infos:
        raw_type = item.get("type", "GeneralTopic")
        gen_nodes.append({
            "id": generate_spot_id("VN", item.get("name")),
            "name": item.get("name"),
            "original_type": raw_type,
            "content": clean_text(item.get("content")),
            "embedding_text": clean_text(item.get("embedding_text")),
            "image_path": clean_text(item.get("image_path"))
        })

    # B. Dữ liệu Provinces
    provinces = data.get("Provinces", [])
    print(f"🔸 Tìm thấy {len(provinces)} Tỉnh/Thành.")
    
    prov_nodes = []
    for province in provinces:
        p_id = province.get("id")     # VD: P_000
        p_name = province.get("name") # VD: HÀ NỘI
        items = province.get("items", [])
        
        for item in items:
            raw_type = item.get("type", "Place")
            item_name = item.get("name", "Unknown")
            
            prov_nodes.append({
                "id": generate_spot_id(p_id, item_name),
                "name": item_name,
                "province_id": p_id,
                "province_name": p_name,
                "original_type": raw_type,
                "neo4j_label": LABEL_MAP.get(raw_type, "TouristDestination"),
                "content": clean_text(item.get("content")),
                "embedding_text": clean_text(item.get("embedding_text")),
                "image_path": clean_text(item.get("image_path"))
            })

    total_nodes = len(gen_nodes) + len(prov_nodes)
    print(f"📊 Tổng số lượng Node cần nạp: {total_nodes}")

    # --- 3. THỰC THI TRÊN NEO4J ---
    if total_nodes > 0:
        with driver.session() as session:
            # Bước 1: Setup Constraints (Đã fix lỗi cú pháp)
            setup_constraints(session)
            
            # Bước 2: Nạp General Info
            if gen_nodes:
                print("⏳ Đang nạp General Info (Quốc gia)...")
                try:
                    session.execute_write(import_general_info, gen_nodes)
                except Exception as e:
                    print(f"❌ Lỗi khi nạp General Info: {e}")

            # Bước 3: Nạp Provinces (Batching)
            if prov_nodes:
                batch_size = 500
                total = len(prov_nodes)
                print(f"⏳ Đang nạp {total} địa điểm thuộc các Tỉnh...")
                
                start_time = time.time()
                for i in range(0, total, batch_size):
                    batch = prov_nodes[i : i + batch_size]
                    try:
                        session.execute_write(import_provinces, batch)
                        print(f"   ✅ Batch {i} -> {min(i+batch_size, total)} OK")
                    except Exception as e:
                        print(f"   ❌ Lỗi tại batch {i}: {e}")
                
                duration = time.time() - start_time
                print(f"⏱️ Thời gian nạp Provinces: {duration:.2f} giây")

    driver.close()
    print("🎉 HOÀN TẤT TOÀN BỘ! Dữ liệu đã lên Neo4j thành công.")

if __name__ == "__main__":
    main()

🚀 Bắt đầu xử lý file: ../data/VanHoaVaDuLichVN_Full_Img.json
🔹 Tìm thấy 6 mục General Information.
🔸 Tìm thấy 63 Tỉnh/Thành.
📊 Tổng số lượng Node cần nạp: 1478
⚙️  Đang cấu hình Constraints (Ràng buộc dữ liệu)...
   ✅ Đã thực thi: CREATE CONSTRAINT FOR (c:Country) REQUIRE c.id IS UNIQUE
   ✅ Đã thực thi: CREATE CONSTRAINT FOR (p:Province) REQUIRE p.id IS UNIQUE
   ✅ Đã thực thi: CREATE CONSTRAINT FOR (n:Searchable) REQUIRE n.id IS UNIQUE
   ✅ Đã thực thi: CREATE CONSTRAINT FOR (g:GeneralInfo) REQUIRE g.id IS UNIQUE
⏳ Đang nạp General Info (Quốc gia)...
⏳ Đang nạp 1472 địa điểm thuộc các Tỉnh...
   ✅ Batch 0 -> 500 OK
   ✅ Batch 500 -> 1000 OK
   ✅ Batch 1000 -> 1472 OK
⏱️ Thời gian nạp Provinces: 1.53 giây
🎉 HOÀN TẤT TOÀN BỘ! Dữ liệu đã lên Neo4j thành công.
